# Introduction

- Import CSV as DataFrame
- Data Cleaning
- Rate Teams
- Rank Teams

### Imports

In [47]:
import pandas as pd
import numpy as np
import trueskill
from trueskill import Rating

import warnings
warnings.filterwarnings("ignore")

In [2]:
# read in the CSGO_TEAMS.csv we saved from the web scrapper
df = pd.read_csv('CSGO_TEAMS.csv')

In [3]:
df.isnull().sum()

Date     136
Team1    136
Team2    136
Map      136
Event    136
dtype: int64

In [4]:
# this drops all NaNs
df = df.dropna()

In [5]:
# double check for null
df.isnull().sum()

Date     0
Team1    0
Team2    0
Map      0
Event    0
dtype: int64

#### Grabbing the rounds won 
- Here we split the number of rounds won from the team name
- Create a new column for "Team1" rounds won
- We also need the Team name without the rounds won attached to it. 

#### Team1

In [6]:
df.head()

,Date,Team1,Team2,Map,Event
0,10/5/20,Polar Ace (16),Under 21 (14),nuke Nuke,Mythic Cup 6
1,10/5/20,Under 21 (8),Polar Ace (16),inf Inferno,Mythic Cup 6
2,10/5/20,Big Frames (6),New England Whalers (16),d2 Dust2,Mythic Cup 6
3,10/5/20,Big Frames (5),New England Whalers (16),nuke Nuke,Mythic Cup 6
4,10/5/20,Swedish Canadians (14),Morning Light (16),ovp Overpass,Mythic Cup 6


In [7]:
str_digits = (
df['Team1'].str.strip() # kill extra whitespace
           .str.replace(')','') # replace ) with nothing
           .str.split('(') # split into two: name, number
           .apply(lambda x: x[1].strip()) # kill extra whitespace on the number
)

key = ~str_digits.apply(lambda x: x.isdigit()) # is it not a digit?
str_digits.loc[key] = '-9999' # replace non-numericals with -9999

df['Round1'] = str_digits.astype(np.int) # convert to numerical

In [8]:
df['Team1'] = (df['Team1'].str.strip() # kill extra whitespace
           .str.replace(')','') # kill )
           .str.split('(') # split into two: name, number
           .apply(lambda x: x[0].strip())) # kill extra whitespace on the name

In [9]:
df.head()

,Date,Team1,Team2,Map,Event,Round1
0,10/5/20,Polar Ace,Under 21 (14),nuke Nuke,Mythic Cup 6,16
1,10/5/20,Under 21,Polar Ace (16),inf Inferno,Mythic Cup 6,8
2,10/5/20,Big Frames,New England Whalers (16),d2 Dust2,Mythic Cup 6,6
3,10/5/20,Big Frames,New England Whalers (16),nuke Nuke,Mythic Cup 6,5
4,10/5/20,Swedish Canadians,Morning Light (16),ovp Overpass,Mythic Cup 6,14


#### Team2

In [10]:
str_digits = (
df['Team2'].str.strip() # kill extra whitespace
           .str.replace(')','') # replace ) with nothing
           .str.split('(') # split into two: name, number
           .apply(lambda x: x[1].strip()) # kill extra whitespace on the number
)

key = ~str_digits.apply(lambda x: x.isdigit()) # is it not a digit?
str_digits.loc[key] = '-9999' # replace non-numericals with -9999

df['Round2'] = str_digits.astype(np.int) # convert to numerical

In [11]:
df['Team2'] = (df['Team2'].str.strip() # kill extra whitespace
           .str.replace(')','') # replace )
           .str.split('(') # split into two: name, number
           .apply(lambda x: x[0].strip())) # kill extra whitespace on the name

In [12]:
df.head()

,Date,Team1,Team2,Map,Event,Round1,Round2
0,10/5/20,Polar Ace,Under 21,nuke Nuke,Mythic Cup 6,16,14
1,10/5/20,Under 21,Polar Ace,inf Inferno,Mythic Cup 6,8,16
2,10/5/20,Big Frames,New England Whalers,d2 Dust2,Mythic Cup 6,6,16
3,10/5/20,Big Frames,New England Whalers,nuke Nuke,Mythic Cup 6,5,16
4,10/5/20,Swedish Canadians,Morning Light,ovp Overpass,Mythic Cup 6,14,16


In [13]:
# grabbing map name without abbreviation
df['Map'] = df['Map'].str.split(' ').str[2].str.split('_').str[0]

In [14]:
df['Map'].unique()

array(['Nuke', 'Inferno', 'Dust2', 'Overpass', 'Vertigo', 'Train',
       'Mirage', 'Cobblestone', 'Cache', 'Tuscan', 'Season', 'Mill',
       'Dust'], dtype=object)

In [15]:
df.head()

,Date,Team1,Team2,Map,Event,Round1,Round2
0,10/5/20,Polar Ace,Under 21,Nuke,Mythic Cup 6,16,14
1,10/5/20,Under 21,Polar Ace,Inferno,Mythic Cup 6,8,16
2,10/5/20,Big Frames,New England Whalers,Dust2,Mythic Cup 6,6,16
3,10/5/20,Big Frames,New England Whalers,Nuke,Mythic Cup 6,5,16
4,10/5/20,Swedish Canadians,Morning Light,Overpass,Mythic Cup 6,14,16


# Team Wins and Loses

### Here we create a column of Team1 and Team2 wins/lose
- For TrueSkill
- Win = 0
- Lose = 1
- Draw = 0

In [16]:
# here we set a bool and convert into an int.
# win1 = Team1 victory
# win2 = Team2 victory
df['Win1'] = (df['Round1'] < df['Round2']).astype(int)
df['Win2'] = (df['Round1'] > df['Round2']).astype(int)

# In TrueSkill
# Win = 0
# Loss = 1
# Draw = 0

In [17]:
df
# From the DataFrame, we can see that a victory is 0
# Loses are 1

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2
0,10/5/20,Polar Ace,Under 21,Nuke,Mythic Cup 6,16,14,0,1
1,10/5/20,Under 21,Polar Ace,Inferno,Mythic Cup 6,8,16,1,0
2,10/5/20,Big Frames,New England Whalers,Dust2,Mythic Cup 6,6,16,1,0
3,10/5/20,Big Frames,New England Whalers,Nuke,Mythic Cup 6,5,16,1,0
4,10/5/20,Swedish Canadians,Morning Light,Overpass,Mythic Cup 6,14,16,1,0
...,...,...,...,...,...,...,...,...,...
77796,21/9/12,NiP,34united,Dust2,DreamHack Valencia 2012,16,2,0,1
77797,13/9/12,FMESPORTS,Blight,Dust2,CyberGamer Qualifier Cup #1 by Tt eSPORTS,7,16,1,0
77798,13/9/12,Blight,FMESPORTS,Inferno,CyberGamer Qualifier Cup #1 by Tt eSPORTS,16,2,0,1
77799,13/9/12,NiP,PRiME,Mirage,Go4CS:GO Cup #6,16,1,0,1


#### Checking amounts of unique team names on each columns
- Check Team1
- Check Team2

In [18]:
df["Team1"].value_counts()

Virtus.pro       921
mousesports      859
fnatic           841
NiP              821
Natus Vincere    715
                ... 
:thinking:         1
zuom               1
n0rth reb0rn       1
PEEKERS            1
yodelin            1
Name: Team1, Length: 4064, dtype: int64

In [19]:
df["Team2"].value_counts()

Virtus.pro            856
NiP                   840
fnatic                817
Natus Vincere         764
HellRaisers           734
                     ... 
GG1337                  1
Anxiety                 1
Wiadro-Gaming           1
Toggle When Needed      1
FTW. Prinfor            1
Name: Team2, Length: 4199, dtype: int64

In [20]:
t1wins = df.groupby('Team1')[['Win1']].agg(['sum', 'count']).reset_index()
t1wins.head()

Team1 Win1      
              sum count
0   #FREEIBP    1     6
1      #SKAM    0     1
2  $HAMELE$$    1     2
3      -0_o-    2     2
4       .4sv    2     2

In [84]:
t2wins = df.groupby('Team2')[['Win2']].agg(['sum', 'count']).reset_index()
t2wins.Win2.sort_values(by='sum', ascending=False)

,sum,count
3655,828,1777
4351,668,1584
1584,662,1418
4186,637,1658
2332,626,1661
...,...,...
2406,0,3
169,0,2
3323,0,1
3324,0,1


In [68]:
t2wins.Win2['sum']

0       2
1       2
2       3
3       2
4       2
       ..
4626    1
4627    6
4628    5
4629    4
4630    4
Name: sum, Length: 4631, dtype: int64

### It looks like our Team name columns have different amount of unique teams. 
- One way to take care of this problem. Append the list of team name to the other column of team names.
- So, append team2 column to team1 and vice versa
- This way the dataframe will have the same amount of team names and rows.

#### Creating a dictionary to swap Column names and also it's contents
- After renaming the columns we concatinate the dataframe 
- This will balance the dataframes columns.

In [22]:
# create a shallow copy of our dataframe
# create a list of the column names


temp = df.copy()
cols = df.columns.tolist()

# select which columns to rename
_dict = {'Team1':'Team2', 'Team2':'Team1', 'Win1':'Win2', 'Win2':'Win1', 'Round1':'Round2', 'Round2':'Round1'}

# renaming the columns from the copied dataframe
temp = temp.rename(_dict, axis=1)

# concat the dataframes, same format as columns.tolist()
df = pd.concat([df[cols], temp[cols]])

# dumping trash
import gc
gc.enable()

del temp; gc.collect()

21

In [23]:
# after concatinating, reset the index
df.reset_index(inplace=True, drop=True)
df.tail()

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2
155325,21/9/12,34united,NiP,Dust2,DreamHack Valencia 2012,2,16,1,0
155326,13/9/12,Blight,FMESPORTS,Dust2,CyberGamer Qualifier Cup #1 by Tt eSPORTS,16,7,0,1
155327,13/9/12,FMESPORTS,Blight,Inferno,CyberGamer Qualifier Cup #1 by Tt eSPORTS,2,16,1,0
155328,13/9/12,PRiME,NiP,Mirage,Go4CS:GO Cup #6,1,16,1,0
155329,13/9/12,PRiME,NiP,Inferno,Go4CS:GO Cup #6,3,16,1,0


In [24]:
# Since then index is not the best way to order the time line of matches
# It will be best to use the "Date"
# Convert the "Date" column into datetime
# Sort the dataframe by the "Date"

df['Date'] = pd.to_datetime(df.Date)
df.sort_values(by='Date', inplace=True)

In [25]:
# After sorting by the "Date", we need to reset the index as well
df.reset_index(inplace=True, drop=True)
df

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2
0,2012-01-11,CPLAY,dotpiXels,Mirage,ESL Pro Series Germany Winter Season 2012,6,16,1,0
1,2012-01-11,dotpiXels,CPLAY,Mirage,ESL Pro Series Germany Winter Season 2012,16,6,0,1
2,2012-01-11,iPLAY,gamed!de,Train,ESL Pro Series Germany Winter Season 2012,7,16,1,0
3,2012-01-11,ALTERNATE aTTaX,CPLAY,Train,ESL Pro Series Germany Winter Season 2012,16,9,0,1
4,2012-01-11,CPLAY,ALTERNATE aTTaX,Train,ESL Pro Series Germany Winter Season 2012,9,16,1,0
...,...,...,...,...,...,...,...,...,...
155325,2020-12-04,MIBR,HAVU,Vertigo,Flashpoint 1,8,16,1,0
155326,2020-12-04,Spirit,BIG,Mirage,Home Sweet Home Cup,4,16,1,0
155327,2020-12-04,Spirit,BIG,Inferno,Home Sweet Home Cup,16,10,0,1
155328,2020-12-04,Espada,Hard Legion,Mirage,LOOT.BET Season 6,22,25,1,0


### TrueSkill Rating Teams
- Initially we need to give teams a base Mu and Sigma
- Mu=25 (rating)
- Sigma=8.33 (confidence)
- TrueSkill standard mu=25 and sigma=8.33 (similar to Halo rating)

In [26]:
# Grab the unique team1 names, convert into a list
# create a new rank list
# loop through the teams give them all the same Mu and Sigma
# this will be the teams initial ratings before we apply the wins
# form a dictionary and zip the team names with their base rating/sigma
# team name: rating, sigma

team_names = df['Team1'].unique().tolist()
ts = trueskill.TrueSkill()

rating = []

for team in team_names:
    rating.append(ts.create_rating())
    
rating_dict = dict(zip(team_names, rating))
rating_dict

{'CPLAY': trueskill.Rating(mu=25.000, sigma=8.333),
 'dotpiXels': trueskill.Rating(mu=25.000, sigma=8.333),
 'iPLAY': trueskill.Rating(mu=25.000, sigma=8.333),
 'ALTERNATE aTTaX': trueskill.Rating(mu=25.000, sigma=8.333),
 'gamed!de': trueskill.Rating(mu=25.000, sigma=8.333),
 'eXtensive': trueskill.Rating(mu=25.000, sigma=8.333),
 'k1ck': trueskill.Rating(mu=25.000, sigma=8.333),
 'VeryGames': trueskill.Rating(mu=25.000, sigma=8.333),
 'myDGB.net': trueskill.Rating(mu=25.000, sigma=8.333),
 'dAT': trueskill.Rating(mu=25.000, sigma=8.333),
 'NiP': trueskill.Rating(mu=25.000, sigma=8.333),
 'CPH Wolves': trueskill.Rating(mu=25.000, sigma=8.333),
 'ESC': trueskill.Rating(mu=25.000, sigma=8.333),
 'Anexis': trueskill.Rating(mu=25.000, sigma=8.333),
 'BuyKey': trueskill.Rating(mu=25.000, sigma=8.333),
 'Brussels Guardians': trueskill.Rating(mu=25.000, sigma=8.333),
 'zNation': trueskill.Rating(mu=25.000, sigma=8.333),
 'ROCCAT': trueskill.Rating(mu=25.000, sigma=8.333),
 'HastaLaVista': tr

In [27]:
# We grab the array of team names from Team1 and Team2
# Then stack them to make an array of arrays
# This way we can match up the two teams that are playing against each other
team1_values = df['Team1'].values
team2_values = df['Team2'].values
matches_array = np.stack((team1_values, team2_values), axis=-1)

In [28]:
matches_array

array([['CPLAY', 'dotpiXels'],
       ['dotpiXels', 'CPLAY'],
       ['iPLAY', 'gamed!de'],
       ...,
       ['Spirit', 'BIG'],
       ['Espada', 'Hard Legion'],
       ['Movistar Riders', 'Heretics']], dtype=object)

In [29]:
# We grab the array of win for team1 and team2
# Then stack them to make an array of arrays
# This way we know who won and who lose
t1_wins = df['Win1'].values
t2_wins = df['Win2'].values
match_out = np.stack((t1_wins, t2_wins), axis=-1)

In [30]:
match_out

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [1, 0]])

### Applying win/lose record to teams
- Now that we have all the teams rating set up, we need to rank them.
- Here we will account for the matches that the teams won and lose.
- To get the most information, we will have a column for their rating before the match and after the match.

In [31]:
# Here we creat a function for rating
def rating(mu, sigma):
    return mu-3*sigma

In [32]:
# Creating two list
# Teams current rating before match and rating after match
curr_ranks = []
after_ranks = []

for m in range(len(matches_array)):
    
# select our dictionary of team base ratings
# looping teams from match array, selecting index 0(team1) and 1(team2)
# assign them to variable
# rating the team's Mu and Sigma will return their rank
# appending rank to list curr_rank
# now we have a list of teams' rank before match(current ranks)
    team1_rating = rating_dict[matches_array[m][0]]
    team2_rating = rating_dict[matches_array[m][1]]
    
    curr_ranks.append([rating(team1_rating.mu, team1_rating.sigma),
                     rating(team2_rating.mu, team2_rating.sigma)])
    
# assigning team ratings 
# setting up for TrueSkill rate method
# assign var. for teams' new rank after the match

    new_rank = ts.rate([(team1_rating,),  (team2_rating,)], ranks= match_out[m])
    
    # grabing team1 and team2 rankings
    new_team1_rank = new_rank[0][0]
    new_team2_rank = new_rank[1][0]
    
    # rating teams' new Mu and Sigma rank
    
    after_ranks.append([rating(new_team1_rank.mu, new_team1_rank.sigma),
                       rating(new_team2_rank.mu, new_team2_rank.sigma)])
    
    rating_dict[matches_array[m][0]] = new_team1_rank
    rating_dict[matches_array[m][1]] = new_team2_rank

In [33]:
df = pd.concat([df, pd.DataFrame(curr_ranks, columns= ['Team1_rank_before', 'Team2_rank_before']),
               pd.DataFrame(after_ranks, columns= ['Team1_rank_after', 'Team2_rank_after'])], axis=1)

In [34]:
df.head()

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2,Team1_rank_before,Team2_rank_before,Team1_rank_after,Team2_rank_after
0,2012-01-11,CPLAY,dotpiXels,Mirage,ESL Pro Series Germany Winter Season 2012,6,16,1,0,0.000000,0.000000,-0.910259,7.881405
1,2012-01-11,dotpiXels,CPLAY,Mirage,ESL Pro Series Germany Winter Season 2012,16,6,0,1,7.881405,-0.910259,11.659386,-0.799872
2,2012-01-11,iPLAY,gamed!de,Train,ESL Pro Series Germany Winter Season 2012,7,16,1,0,0.000000,0.000000,-0.910259,7.881405
3,2012-01-11,ALTERNATE aTTaX,CPLAY,Train,ESL Pro Series Germany Winter Season 2012,16,9,0,1,0.000000,-0.799872,6.395132,-1.119980
4,2012-01-11,CPLAY,ALTERNATE aTTaX,Train,ESL Pro Series Germany Winter Season 2012,9,16,1,0,-1.119980,6.395132,-1.074714,9.716758


#### Rating Teams to get Rank
- Here we rate the teams' Mu and Sigma to return a Rank 

In [35]:
ranks = pd.DataFrame(rating_dict).transpose()
ranks.columns = ['mu', 'sigma']
ranks['rank'] = rating(ranks['mu'], ranks['sigma'])
ranks = ranks.sort_values(by='rank', ascending=False)

In [36]:
# there are Staff teams in our ranks dataframe
# lets remove them
ranks

,mu,sigma,rank
Astralis,38.049462,0.798274,35.654638
HLTV.org Staff,40.170944,1.675748,35.143699
Liquid,37.290050,0.798710,34.893920
NRG,37.199413,0.811391,34.765239
FaZe,36.894034,0.809491,34.465562
...,...,...,...
Vapebar,7.841713,4.115049,-4.503432
UnifyHK,10.706348,5.130204,-4.684264
LowLandLions 1.6,7.877001,4.224927,-4.797782
OK.Blaze,10.202819,5.074811,-5.021612


In [37]:
# create a list of teams names that have "Staff" in them
staffs = [team for team in df['Team1'].unique().tolist() if 'Staff' in team]
staffs

['Russian eSF Staff',
 'HLTV.org Staff',
 'Hitbox.tv Staff',
 'Game Show Staff',
 'Virtus.pro Staff',
 'SLTV Staff',
 'LC Staff',
 'Na`Vi Staff',
 'Vakarm.net Staff',
 'NiP Staff',
 'G2A.com Staff',
 'Fragbite.se Staff',
 'Cybersport.pl Staff',
 'myXMG Staff',
 'PGL Staff',
 'FACEIT Staff',
 'Gamers.com.mt Staff',
 'StormStudio Staff',
 'E-frag.net Staff',
 'Fnatic Staff',
 'EGB.com Staff',
 '99damage.de Staff']

In [38]:
# new rank dataframe without the Staff teams
ranks = ranks[~ranks.index.isin(staffs)]

In [39]:
# reset the index and rename column of teams to "team_name"
ranks.reset_index(inplace=True)
ranks.rename(columns={'index': 'team_name'}, inplace=True)

In [40]:
# now we have our ranks dataframe completed
ranks.head()

,team_name,mu,sigma,rank
0,Astralis,38.049462,0.798274,35.654638
1,Liquid,37.290050,0.798710,34.893920
2,NRG,37.199413,0.811391,34.765239
3,FaZe,36.894034,0.809491,34.465562
4,fnatic,36.780262,0.784274,34.427441


In [41]:
# looks like our dataframe has Staff teams playing in matches
# lets remove them from our original dataframe as well
df[df['Team1'] == 'HLTV.org Staff']

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2,Team1_rank_before,Team2_rank_before,Team1_rank_after,Team2_rank_after
9927,2014-11-15,HLTV.org Staff,LC Staff,Cache,Virtus.pro Staff Cup #1,16,12,0,1,0.000000,0.000000,7.881405,-0.910259
9936,2014-11-15,HLTV.org Staff,SLTV Staff,Cache,Virtus.pro Staff Cup #1,16,11,0,1,14.034964,7.881405,18.113546,7.626408
9945,2014-11-15,HLTV.org Staff,Russian eSF Staff,Dust2,Virtus.pro Staff Cup #1,16,0,0,1,19.390368,-1.268278,19.954564,-1.079902
9946,2014-11-15,HLTV.org Staff,Virtus.pro Staff,Dust2,Virtus.pro Staff Cup #1,16,5,0,1,19.954564,5.734756,21.179683,5.924092
9999,2014-11-23,HLTV.org Staff,NiP Staff,Mirage,Virtus.pro Staff Cup #1,16,13,0,1,24.661680,0.910227,25.354349,1.549354
10000,2014-11-23,HLTV.org Staff,NiP Staff,Cache,Virtus.pro Staff Cup #1,16,7,0,1,25.354349,1.549354,25.892969,2.028250
10003,2014-11-23,HLTV.org Staff,NiP Staff,Overpass,Virtus.pro Staff Cup #1,16,14,0,1,26.693558,2.710677,27.004545,2.966425
22725,2015-11-13,HLTV.org Staff,LC Staff,Cobblestone,Virtus.pro Staff Cup #3,14,16,1,0,28.150523,21.456436,25.726807,23.821587
22726,2015-11-13,HLTV.org Staff,LC Staff,Cache,Virtus.pro Staff Cup #3,16,6,0,1,25.726807,23.821587,27.160687,23.607834
22748,2015-11-13,HLTV.org Staff,LC Staff,Mirage,Virtus.pro Staff Cup #3,16,5,0,1,27.160687,23.607834,28.237355,23.455452


In [85]:
# new dataframe without Staff teams
# now we have a completed dataframe
df = df[~df.Team1.isin(staffs)]
df = df[~df.Team2.isin(staffs)]
df.head()

,Date,Team1,Team2,Map,Event,Round1,Round2,Win1,Win2,Team1_rank_before,Team2_rank_before,Team1_rank_after,Team2_rank_after
0,2012-01-11,CPLAY,dotpiXels,Mirage,ESL Pro Series Germany Winter Season 2012,6,16,1,0,0.000000,0.000000,-0.910259,7.881405
1,2012-01-11,dotpiXels,CPLAY,Mirage,ESL Pro Series Germany Winter Season 2012,16,6,0,1,7.881405,-0.910259,11.659386,-0.799872
2,2012-01-11,iPLAY,gamed!de,Train,ESL Pro Series Germany Winter Season 2012,7,16,1,0,0.000000,0.000000,-0.910259,7.881405
3,2012-01-11,ALTERNATE aTTaX,CPLAY,Train,ESL Pro Series Germany Winter Season 2012,16,9,0,1,0.000000,-0.799872,6.395132,-1.119980
4,2012-01-11,CPLAY,ALTERNATE aTTaX,Train,ESL Pro Series Germany Winter Season 2012,9,16,1,0,-1.119980,6.395132,-1.074714,9.716758


In [86]:
# save new dataframe, 
df.to_csv(r'/Users/bi/Documents/CSGO_Win_Probability/CSGO_TEAMS_MATCHES.csv', index=False)

In [44]:
# save team rank dataframe
ranks.to_csv(r'/Users/bi/Documents/CSGO_Win_Probability/CSGO_TEAMS_RANKS.csv', index=False)

### Summary
- Removed Nans
- Created columns for rounds won by each team
- Classified the wins and loses for each team in a new column
    - Based on TrueSkill rules (win=0, lose=1, draw=0)
- Balanced the DataFrame rows because we had different amounts of unique Team names

#### TrueSkill Rating
- Used TrueSkill to give teams a base Mu and Sigma (mu=25, sigma=8.33)
- Collected arrays of Team name in a match and also their win/lose
- Rated the team based on their win/lose for each match
- Saved their before and after ratings for each match
- Then we ranked the teams based on their new Mu and Sigma

#### Organized and Sort
- We then organized the team names by removing Staff teams
- Removed Staff teams from Rank and Original dataframe
- Save both dataframe for modeling